In [1]:
#구글드라이브와 연결
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# **1. 네이버**

In [0]:
import csv
import requests
from bs4 import BeautifulSoup
from collections import OrderedDict
from itertools import count #count 불러오기
import re
from google.colab import files


 
def rednooby_cralwler(input_search):
    global filename
    filename=input_search
    f = open('/content/gdrive/My Drive/Colab Notebooks/'+filename+'_n.csv', 'w', encoding='utf-8', newline='')
    wr = csv.writer(f)
    url = 'https://search.naver.com/search.naver'
    post_dict = OrderedDict()
    
    for page in count(1):#1부터 무한대로 시작(break or return이 나올때까지)
        params = {
            'query': input_search,
            'where': 'post',
            'start': (page-1)*10+1,
        }
        print(params)
        response = requests.get(url, params=params)
        html = response.text
 
        #뷰티풀소프의 인자값 지정
        soup = BeautifulSoup(html, 'html.parser')
 
        #쪼개기
        title_list = soup.select('.sh_blog_title')
 
        for tag in title_list:
            if tag['href'] in post_dict:
                return post_dict#여기 오게되면 count는 종료됩니다.

            #네이버 블로그에서 검색한 링크로 들어갈 경우 html구조가 다름
            #따라서 페이지를 경유해서 들어가지 않을 때와 같은 기존의 페이지 링크로 변경하는 과정
            #제대로된 html문서를 위해 필요한 과정

            try:
               url_1 = tag['href']
               html_result = requests.get(url_1)
               soup_temp = BeautifulSoup(html_result.text, 'html.parser')
               area_temp= soup_temp.find(id='screenFrame')
               url_2 = area_temp.get('src')
            except:
               try:
                  area_temp= soup_temp.find(id='mainFrame')
                  url_3 = area_temp.get('src')
                  url_4 = "http://blog.naver.com"+url_3
               except :
                  return None
            try:
               html_result =requests.get(url_2)
               soup_temp = BeautifulSoup(html_result.text,'html.parser')
               area_temp = soup_temp.find(id='mainFrame')
               url_3 = area_temp.get('src')
               url_4 = "http://blog.naver.com"+url_3
               return url_4
            except :
               pass
            print(tag.text, url_4)

            post_dict[tag['href']] = tag.text
            p_rec_contents = requests.get(url_4)
            psoup = BeautifulSoup(p_rec_contents.content,'html.parser')
            ptemp=psoup.findAll("div",{"class":"se-main-container"}) #네이버 블로그의 포스팅 본문부분
            #포스팅 본문을 한줄로 출력하기
            array =[]
            for a in ptemp:
              if(a.get_text()!='\n'or a.get_text()!=' '):
                 text =a.get_text().replace("\n","")  #\n 제거
                 text =text.replace("  ","")        #공백 두개일 경우 제거
                 text =text.replace("\u200b","")    #이부분은 출력할때 나오는 이상한 부분 제거하는 거
                 array.append(text)
            
            print("-----[post내용]-----")
            array_string="".join(array)   
            result_post = find_sentence(array_string) #포스팅 본문을 문장으로 나누는 함수 호출
            print(result_post)
            print('\n')
            
            wr.writerow([url_4, result_post])   #csv파일에 [링크,본문] 형태로 추가
            
    f.close()

    return post_dict

#문장을 찾는 함수
def find_sentence(sentences):
  list_dot = sentences.split('.')  #문장을 '.'으로 나눔
  list_excla =[]
  list_result =[]
  for dot_sen in list_dot:
    list_excla.append(dot_sen.split('!'))  #문장을 '!'으로 나눔
    
  for excla_sen in list_excla:
    for result_sen in excla_sen:
      list_result.append(result_sen.split('?'))   #문장을 '?'으로 나눔
  list_result=sum(list_result, [])    #sum을 쓰는 이유는 다중리스트를 flatten하게 만들기 위함
  return list_result

target = input('검색어를 입력하세요:')
rednooby_cralwler(target)

검색어를 입력하세요:웹 개발자
{'query': '웹 개발자', 'where': 'post', 'start': 1}
웹개발자연봉, 프론트엔드 백엔드의 차이는? http://blog.naver.com/PostView.nhn?blogId=cdh0584&logNo=221806379140&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
-----[post내용]-----
['안녕하세요IT 읽어주는 여자, 예티입니다프론트엔드, 그리고 백엔드 개발자에 대해서많이 들어보신 분들도 있을거에요', '웹 개발이라는 하나의 큰 분야에서포지션을 2가지로 나눴다고 보시면 되는데간단히 요약해서 말씀드리면', '', ' 프론트엔드 = 웹 디자인백엔드 = 웹구축 및 서버,DB연동라고 생각하시면 됩니다', ' 같은 웹 개발자에 속하지만프론트엔드와 백엔드 차이를 명확히 모르실 수 있기 때문에 두가지 직군이 취업하게 되면 무슨일을 하게 될지 차이를 알아보고연봉은 대략 어느정도 되는지 알아볼게요', ' 프론트엔드란', ' [웹디자이너 = 웹퍼블리셔 = 프론트엔드]웹 사이트(naver/daum 등)의 사용자에게 보여지는 시각적인 부분들을 담당', '즉, 사용자에게 보여지는 사이트의 디자인을 구현하는 역할을 맡고 있어요', ' 웹디자이너는 코딩을 주로 다루는 비중이 크므로,꼭 미술감각이 있어야 하는것은 아님', '-가장 많이묻는 질문의 답-이렇게 말해도 잘 이해가 안될 수 있어서네이버를 예시로 들어볼게요', ' (좌)네이버 구버전/ (우)네이버 현재 버전 예전과 얼핏 비슷하지만 많이 달라진 네이버의 모습이 보이시나요', '검색창 디자인과 상단의 카테고리들, 간편화된 로그인창과 쇼핑·언론사배너 등네이버 메인 디자인이 변화된 것과 구성위치가 바뀐 부분들을 볼 수 있죠', '이렇게 사용자에게 보여지는 웹 사이트 화면을 구성하는 영역들(=그림/레이아웃/창 배치 등)을 수정하고만드는것을 담당하는 직군이 프론트엔드 랍니다', ' 프론트엔드가 다루

In [0]:
#CSV 파일 PC에 다운로드
!ls
files.download('/content/gdrive/My Drive/Colab Notebooks/'+filename+'_n.csv')

gdrive	sample_data


# **<구글>**

In [2]:
# -*- encoding: utf-8 -*-
from collections import Counter
from bs4 import BeautifulSoup as BS
from google.colab import files
import ssl, urllib 
import traceback
import re
import csv
import pandas as pd
import os.path

def DivSenDot(matchObj):
    return matchObj.group().replace('.', '._\n')

#불필요한 요소들을 제거하는 함수
def test(site_list):
  site_list = re.search('<p.*</p>', str(site_list), re.I|re.S)
  site_list = site_list.group()
  site_list = re.sub('<script.*?>.*?</script>', '', site_list, 0, re.I|re.S)
  site_list = re.sub('<style.*?>.*?</style>', '', site_list, 0, re.I|re.S)
  site_list = re.sub('<footer.*?>.*?</footer>', '', site_list, 0, re.I|re.S)

  site_list = re.sub('<.+?>', '', site_list, 0, re.I|re.S)
  site_list = re.sub(r'\[[^)]*\]', '', site_list,0, re.I|re.S)
  site_list = re.sub(pattern=r'[\u3131-\u3163\uac00-\ud7a3][.]',repl=DivSenDot,string=site_list)

  site_list = site_list.replace('?', '?\n').replace('!', '!\n').replace(',',' ').replace("\xa0"," ").replace("&amp;nbsp"," ")
  site_list = site_list.replace("\r","\n").replace('\t','\n').replace('\r\n','\n')
  site_list = site_list.strip()
  site_list = "".join([s for s in site_list.strip().splitlines(True) if s.strip()]) #문자열로 만들어줌
  return site_list

#문장을 찾는 함수
def find_sentence(sentences):
  list_dot = sentences.split('._')  #문장을 '.'으로 나눔
  list_excla =[]
  list_result =[]
  for dot_sen in list_dot:
    list_excla.append(dot_sen.split('!'))  #문장을 '!'으로 나눔
    
  for excla_sen in list_excla:
    for result_sen in excla_sen:
      list_result.append(result_sen.split('?'))   #문장을 '?'으로 나눔
  list_result=sum(list_result, [])    #sum을 쓰는 이유는 다중리스트를 flatten하게 만들기 위함
  return list_result

google_url_list = [] #검색 결과 구글 링크 주소들의 배열
title_list = []      #검색 결과 구글 타이틀들의 배열
base_url = 'https://www.google.co.kr/search' 
target = input('검색어를 입력하세요:')
file = '/content/gdrive/My Drive/Colab Notebooks/'+target+'_g.csv'

#: 검색조건 설정 
values = {
 'q' : target, #검색할 내용
 'oq' : target,
 'aqs': 'chrome..69i57.35694j0j7',
 'sourceid': 'chrome',
 'ie': 'UTF-8',
 } 

# Google에서는 Header 설정 필요 
hdr = {'User-Agent': 'Mozilla/5.0'}

query_string = urllib.parse.urlencode(values) 
req = urllib.request.Request(base_url + '?' + query_string, headers=hdr) 
context = ssl._create_unverified_context()
try: 
  res = urllib.request.urlopen(req, context=context)

except:
  traceback.print_exc() 

html_data = BS(res.read(), 'html.parser')

#검색된 결과에 해당하는 모든 검색 결과 사이트들 주소 저장
while True:
  next_page = html_data.find('a', attrs={'aria-label' : 'Next page'}) #다음 페이지 주소를 찾음 
  g_list = html_data.find_all('div', attrs={'class': 'kCrYT'})        #검색 결과 사이트들을 찾음
  search = "youtube"
  
  try:
    for g in g_list:
      try:
        ahref = g.find('a')['href']
        if search in ahref:
          pass
        else:
          ahref = 'https://www.google.co.kr' + ahref
          google_url_list.append(ahref)   #검색된 링크들을 리스트에 저장
                   
      except TypeError as e:
        pass
  except:
    traceback.print_exc()

  try:
    ahref = next_page.get('href')
    ahref = 'https://www.google.co.kr' + ahref
    req = urllib.request.Request(ahref, headers=hdr)
    context = ssl._create_unverified_context()
    try:
      res = urllib.request.urlopen(req, context=context)
    except:
      traceback.print_exc()
  except AttributeError:
    break  

  html_data = BS(res.read(), 'html.parser')  #다음페이지로 넘어감

def SaveCsv():
  #google_url_list안에 있는 링크 하나씩 가지고 와서 csv 파일 만들기
  for url in google_url_list:
    req = urllib.request.Request(url, headers=hdr)
    context = ssl._create_unverified_context()
    try:
      res = urllib.request.urlopen(req, context=context)
    except:
      traceback.print_exc()

    html_data = BS(res.read(), 'html.parser')

    try:
      site_list = html_data.find_all('p')
      title = html_data.find('title').get_text().strip()
      print(url+" "+title+"\n 파일 만드는 중")

      #같은 검색 결과 구글 타이틀이 없는지 확인
      if title not in title_list: 
        title_list.append(title)    #없다면 해당 타이틀 타이틀 배열에 저장 
    
        text = []
        site_list = test(site_list)
        site_list = site_list.split('\n')
        for t in site_list:
          text.append(t.strip())  #각 배열 원소들의 공백을 지우고 다시 배열에 저장 

        text_string = "".join(text)
        text_string = find_sentence(text_string)

        f = open(file, '+a', encoding = 'utf-8')
        wr = csv.writer(f)
        wr.writerow([url, text_string])  
        f.close()
        print("파일 만듦")   
      else:
        print("해당 제목은 이미 존재하는 제목입니다.")
    except:
      traceback.print_exc() 
      pass

  print("파일 만들기 완성")

#파일 존재하는지 확인
if os.path.isfile(file): #이미 존재
  os.remove(file)
  print("파일을 재생성합니다.")
else : #존재하지 않음
  pass
f = open(file, 'w', encoding = 'utf-8')
f.close()
SaveCsv()

검색어를 입력하세요:웹프로그래밍
https://www.google.co.kr/url?q=https://www.edwith.org/boostcourse-web/lecture/16660/&sa=U&ved=2ahUKEwjcsJOiufPnAhUG7J4KHdewC6cQFjAAegQIARAB&usg=AOvVaw0i4dZ6jiE2C25hO-9TptEL [부스트코스] 웹 프로그래밍 > 1) 웹 프로그래밍을 위한 프로그램 언어들 : edwith
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://brunch.co.kr/%40imagineer/78&sa=U&ved=2ahUKEwjcsJOiufPnAhUG7J4KHdewC6cQFjABegQICBAB&usg=AOvVaw2UpXwoA49XQY-_Tvct2yBC 웹 프로그래밍이 뭔가요?
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=http://m.blog.naver.com/7bangjuni/221254771707&sa=U&ved=2ahUKEwjcsJOiufPnAhUG7J4KHdewC6cQFjACegQICRAB&usg=AOvVaw0DpxDJjrTer13CeeJP8xp3 웹프로그래밍 종류와 공부순서 : 네이버 블로그
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=http://www.bloter.net/archives/176582&sa=U&ved=2ahUKEwjcsJOiufPnAhUG7J4KHdewC6cQFjADegQIChAB&usg=AOvVaw3VABwbnvETt0jZp3zz9CP6 초보에서 고급까지, 웹으로 프로그래밍 배우자 | Bloter.net
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://cloudstudying.kr/lectures/208&sa=U&ved=2ahUKEwjcsJOiufPnAhUG7J4KHdewC6cQFjAEegQIBxAB&usg=AOvVaw001

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Traceback (most recent call last):
  File "<ipython-input-2-9aaccc3773d1>", line 124, in SaveCsv
    title = html_data.find('title').get_text().strip()
AttributeError: 'NoneType' object has no attribute 'get_text'


https://www.google.co.kr/url?q=https://jhnyang.tistory.com/83&sa=U&ved=2ahUKEwiRx82iufPnAhURuZ4KHdZLAfM4ChAWMAR6BAgGEAE&usg=AOvVaw3zj2GNQR7pl9HF6aVrkObZ 프로그래밍 언어 종류(C. Java, php 등 10개)와 사용도, 순위
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://songhyonkim.gitbooks.io/web-programming-howto/content/web.html&sa=U&ved=2ahUKEwiRx82iufPnAhURuZ4KHdZLAfM4ChAWMAV6BAgEEAE&usg=AOvVaw0YMMobzlHOtX7Afo89MiQW 웹 프로그래밍 개요 | 웹 프로그래밍을 위한 넓고 얕은 지식
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://nch1234.tistory.com/1027&sa=U&ved=2ahUKEwiRx82iufPnAhURuZ4KHdZLAfM4ChAWMAZ6BAgCEAE&usg=AOvVaw02Fh8JUji1YBpFgKk2lyD1 사용할 웹 프로그래밍 언어를 선택하는 방법
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://www.aladin.co.kr/shop/wproduct.aspx%3FItemId%3D206059434&sa=U&ved=2ahUKEwiRx82iufPnAhURuZ4KHdZLAfM4ChAWMAd6BAgDEAE&usg=AOvVaw3bA31lnsArb632qcCdQdl4 알라딘: Front-End 웹프로그래밍
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://www.fastcampus.co.kr/dev_school_wps/&sa=U&ved=2ahUKEwiRx82iufPnAhURuZ4KHdZLAfM4ChC3AjAIe

Traceback (most recent call last):
  File "<ipython-input-2-9aaccc3773d1>", line 132, in SaveCsv
    site_list = test(site_list)
  File "<ipython-input-2-9aaccc3773d1>", line 17, in test
    site_list = site_list.group()
AttributeError: 'NoneType' object has no attribute 'group'


https://www.google.co.kr/url?q=https://javarayo.tistory.com/12&sa=U&ved=2ahUKEwiax_aiufPnAhWVoJ4KHe0MDNs4FBAWMAV6BAgHEAE&usg=AOvVaw3lLotYY0YcOjcMVqnpD6pb [1] 웹 프로그래밍이란 :: 앰니저아른이
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://genesis8.tistory.com/231&sa=U&ved=2ahUKEwiax_aiufPnAhWVoJ4KHe0MDNs4FBAWMAZ6BAgCEAE&usg=AOvVaw2s9NtKob4NVnBJbUN_zBHh 프로그래머의 블로그 :: 웹 프로그래밍 기초
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=http://www.csharpstudy.com/web&sa=U&ved=2ahUKEwiax_aiufPnAhWVoJ4KHe0MDNs4FBAWMAd6BAgIEAE&usg=AOvVaw2StB4CS7hIAPO9kEYSk93d C# 웹 프로그래밍 - C# 프로그래밍 배우기 (Learn C# Programming)
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=http://webprogramming.co.kr/&sa=U&ved=2ahUKEwiax_aiufPnAhWVoJ4KHe0MDNs4FBAWMAh6BAgBEAE&usg=AOvVaw0B2YBGj-uTvuwEl06xDP6k 명품 웹 프로그래밍
 파일 만드는 중
파일 만듦


...

https://www.google.co.kr/url?q=https://careers.sorint.it/rjyqw/django-girls-tutorial.html&sa=U&ved=2ahUKEwixuY2mufPnAhWKv54KHVxvAIA4bhAWMAl6BAgDEAE&usg=AOvVaw3AIipe0KjaQDxFkXJvM13P Django girls tutorial
 파일 만드는 중
파일 만듦


Traceback (most recent call last):
  File "<ipython-input-2-9aaccc3773d1>", line 124, in SaveCsv
    title = html_data.find('title').get_text().strip()
AttributeError: 'NoneType' object has no attribute 'get_text'


https://www.google.co.kr/url?q=http://www.kmooc.kr/&sa=U&ved=2ahUKEwj17rqmufPnAhWHq54KHW5sCjM4eBAWMAF6BAgHEAE&usg=AOvVaw2Zage7pNBoYrwsqLtqYhwt | K-MOOC
 파일 만드는 중


Traceback (most recent call last):
  File "<ipython-input-2-9aaccc3773d1>", line 132, in SaveCsv
    site_list = test(site_list)
  File "<ipython-input-2-9aaccc3773d1>", line 17, in test
    site_list = site_list.group()
AttributeError: 'NoneType' object has no attribute 'group'
Traceback (most recent call last):
  File "<ipython-input-2-9aaccc3773d1>", line 124, in SaveCsv
    title = html_data.find('title').get_text().strip()
AttributeError: 'NoneType' object has no attribute 'get_text'


https://www.google.co.kr/url?q=http://dreadv.com.br/home/java-%25EC%259B%25B9%25ED%2594%2584%25EB%25A1%259C%25EA%25B7%25B8%25EB%259E%2598%25EB%25B0%258D-%25EC%2598%2588%25EC%25A0%259C/&sa=U&ved=2ahUKEwj17rqmufPnAhWHq54KHW5sCjM4eBAWMAN6BAgCEAE&usg=AOvVaw1Tpxh_u6DEThiGJV79Pplx java 웹프로그래밍 예제 – David Rechulski Advogados
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=https://www.microsoft.com/ko-kr/software-download/windows10&sa=U&ved=2ahUKEwj17rqmufPnAhWHq54KHW5sCjM4eBAWMAR6BAgEEAE&usg=AOvVaw2S_CSR6FP1DEUQczyLaLON Windows 10 다운로드
 파일 만드는 중
파일 만듦
https://www.google.co.kr/url?q=http://www.ewha.ac.kr/&sa=U&ved=2ahUKEwj17rqmufPnAhWHq54KHW5sCjM4eBAWMAV6BAgDEAE&usg=AOvVaw0bWQP1FJArR-Rc0fOjDmYC 이화여자대학교
 파일 만드는 중
파일 만듦
파일 만들기 완성


In [0]:
#CSV 파일 PC에 다운로드
!ls
files.download('/content/gdrive/My Drive/Colab Notebooks/'+target+'_g.csv')

gdrive	sample_data


구글크롤링은 csv 파일을 그냥 열면 깨지는 문제점이 발생하여서 이를 위해 다음의 절차를 따라야합니다.

1) csv 파일을 메모장으로 열어 다른이름으로 저장을 누릅니다.

2) 파일제목에.csv를 꼭 붙이고 인코딩 형식을 UTF-8에서 ANSI로 바꿔줍니다.

3) 저장을 누릅니다.